### Assignment-01-附加题-极具挑战性的题目，基础一般的同学请把前边的做好就非常非常好了，这个是一个趣味题 😝

## 基于模式匹配的对话机器人实现

### Pattern Match

机器能否实现对话，这个长久以来是衡量机器人是否具有智能的一个重要标志。 Alan Turing早在其文中就提出过一个测试机器智能程度的方法，该方法主要是考察人类是否能够通过对话内容区分对方是机器人还是真正的人类，如果人类无法区分，我们就称之为具有”智能“。而这个测试，后来被大家叫做”图灵测试“，之后也被翻拍成了一步著名电影，叫做《模拟游戏》。 



既然图灵当年以此作为机器是否具备智能的标志，这项任务肯定是复杂的。自从 1960s 开始，诸多科学家就希望从各个方面来解决这个问题，直到如今，都只能解决一部分问题。 目前对话机器人的建立方法有很多，今天的作业中，我们为大家提供一共快速的基于模板的对话机器人配置方式。

此次作业首先希望大家能够读懂这段程序的代码，其次，在此基于我们提供的代码，**能够把它改造成汉语版本，实现对话效果。**

```
Pattern: (我想要A)
Response: (如果你有 A，对你意味着什么呢？)

Input: (我想要度假)
Response: (如果你有度假，对你意味着什么呢？)
```

为了实现模板的判断和定义，我们需要定义一个特殊的符号类型，这个符号类型就叫做"variable"， 这个"variable"用来表示是一个占位符。例如，定义一个目标: "I want X"， 我们可以表示成  "I want ?X", 意思就是?X是一个用来占位的符号。

如果输入了"I want holiday"， 在这里 'holiday' 就是 '?X'

In [ ]:
"""笔记：
一句话总结：（核心）
模式匹配,模式匹配,
模式匹配就是
给定  "I was ?*X"  类型的问题，皆返回回答  "I am agree with you !" 
"""

In [4]:
def is_variable(pat):
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])
#startswith() 方法用于检查字符串是否是以指定子字符串开头
#all() 函数用于判断给定的可迭代参数 iterable 中的所有元素是否都为 TRUE，如果是返回 True，否则返回 False。
#isalpha() 方法检测字符串是否只由字母组成,返回 True或 False。

In [5]:
def pat_match(pattern, saying):
    if is_variable(pattern[0]): return True#意思是如果满足以“？”开头，且“？”后面都是字母，就返回True
    else:
        if pattern[0] != saying[0]: return False#否则就比较首字符串
        else:
            return pat_match(pattern[1:], saying[1:])
#整个函数的意思是，比较连个句子中的索引的单词是否相同，除了“？X”的索引

### 例如

In [5]:
pat_match('I want ?X'.split(), "I want holiday".split())

True

In [6]:
pat_match('I have dreamed a ?X'.split(), "I dreamed about dog".split())

False

In [7]:
pat_match('I dreamed about ?X'.split(), "I dreamed about dog".split())

True

### 获得匹配的变量

以上的函数能够判断两个 pattern 是不是相符，但是我们更加希望的是获得每个variable对应的是什么值。

我们对程序做如下修改:

In [6]:
def pat_match(pattern, saying):
    if is_variable(pattern[0]):
        return pattern[0], saying[0]
    else:
        if pattern[0] != saying[0]: return False
        else:
            return pat_match(pattern[1:], saying[1:])

In [9]:
pattern = 'I want ?X'.split()
saying = "I want holiday".split()

In [10]:
pat_match(pattern, saying)

('?X', 'holiday')

In [11]:
pat_match("?X equals ?X".split(), "2+2 equals 2+2".split())

('?X', '2+2')

但是，如果我们的 Pattern 中具备两个变量，那么以上程序就不能解决了，我们可以对程序做如下修改: 


In [7]:
def pat_match(pattern, saying):
    if not pattern or not saying: return []#这行代码的意思是如果参数的空字符串，就返回一个[]
    
    if is_variable(pattern[0]):
        return [(pattern[0], saying[0])] + pat_match(pattern[1:], saying[1:])
    else:
        if pattern[0] != saying[0]: return []
        else:
            return pat_match(pattern[1:], saying[1:])

In [45]:
pat_match("", "")

[]

于是，我们可以获得： 

In [41]:
pat_match("?X greater than ?Y".split(), "3 greater than 2".split())

[('?X', '3'), ('?Y', '2')]

如果我们知道了每个变量对应的是什么，那么我们就可以很方便的使用我们定义好的模板进行替换：

为了方便接下来的替换工作，我们新建立两个函数，一个是把我们解析出来的结果变成一个 dictionary，一个是依据这个 dictionary 依照我们的定义的方式进行替换。

In [8]:
def pat_to_dict(patterns):
    return {k: v for k, v in patterns}

In [9]:
def subsitite(rule, parsed_rules):
    if not rule: return []
    
    return [parsed_rules.get(rule[0], rule[0])] + subsitite(rule[1:], parsed_rules)

In [123]:
#案例1："I want ?X"
got_patterns = pat_match("I want ?X".split(), "I want iPhone".split())

In [128]:
print((got_patterns))
print(pat_to_dict(got_patterns))

[('?X', 'iPhone')]
{'?X': 'iPhone'}


In [125]:
subsitite("What if you mean if you got a ?X".split(), pat_to_dict(got_patterns))#此时的got_patterns已经被函数封装成字典了
#案例1没写' '.join

['What', 'if', 'you', 'mean', 'if', 'you', 'got', 'a', 'iPhone']

为了将以上输出变成一句话，也很简单，我们使用 Python 的 join 方法即可： 

In [19]:
#案例2：'?P needs ?X'
john_pat = pat_match('?P needs ?X'.split(), "John needs resting".split())#这2句话和上面3句话差不多是一样的

In [20]:
' '.join(subsitite("What if you mean if you got a ?X".split(), pat_to_dict(got_patterns)))#只是多了个' '.join

'What if you mean if you got a iPhone'

In [22]:
john_pat = pat_match('?P needs ?X'.split(), "John needs vacation".split())
john_pat

[('?P', 'John'), ('?X', 'vacation')]

In [23]:
subsitite("Why does ?P need ?X ?".split(), pat_to_dict(john_pat))

['Why', 'does', 'John', 'need', 'vacation', '?']

In [24]:
' '.join(subsitite("Why does ?P need ?X ?".split(), pat_to_dict(john_pat)))
#看到这里，个人觉得基于模式匹配的对话，基本上都是扣关键的名词出来然后displace一下。

'Why does John need vacation ?'

['Image you will get ?X soon', 'Why do you need ?X ?']

那么如果我们现在定义一些patterns，就可以实现基于模板的对话生成了:

In [84]:
#注意，这里和末尾的rule差别是，这里的“？”后面没有“*”
defined_patterns = {
    "I need ?X": ["Image you will get ?X soon", "Why do you need ?X ?"], 
    "My ?X told me something": ["Talk about more about your ?X", "How do you think about your ?X ?"]
}

In [79]:
def get_response(saying, rules):
    """" please implement the code, to get the response as followings:
    
    >>> get_response('I need iPhone') 
    >>> Image you will get iPhone soon
    >>> get_response("My mother told me something")
    >>> Talk about more about your monther.
    """
    pass
    

### Segment Match

我们上边的这种形式，能够进行一些初级的对话了，但是我们的模式逐字逐句匹配的， "I need iPhone" 和 "I need ?X" 可以匹配，但是"I need an iPhone" 和 "I need ?X" 就不匹配了，那怎么办？ 

为了解决这个问题，我们可以新建一个变量类型 "?\*X", 这种类型多了一个星号(\*),表示匹配多个

首先，和前文类似，我们需要定义一个判断是不是匹配多个的variable

In [10]:
def is_pattern_segment(pattern):
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:])

In [11]:
is_pattern_segment('?*P')

True

In [12]:
from collections import defaultdict

In [13]:
def pat_match(pattern, saying):
    if not pattern or not saying: return []#这行代码的意思是如果参数的空字符串，就返回一个[]
    
    if is_variable(pattern[0]):
        return [(pattern[0], saying[0])] + pat_match(pattern[1:], saying[1:])
    else:
        if pattern[0] != saying[0]: return []
        else:
            return pat_match(pattern[1:], saying[1:])

然后我们把之前的 ```pat_match```程序改写成如下， 主要是增加了 ``` is_pattern_segment ```的部分. 

In [14]:
fail = [True, None]

def pat_match_with_seg(pattern, saying):
    if not pattern or not saying: return []
    
    pat = pattern[0]#这行代码是新添的
    
    if is_variable(pat):
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    elif is_pattern_segment(pat):#这行代码是新添的
        match, index = segment_match(pattern, saying)#这行代码是新添的
        return [match] + pat_match_with_seg(pattern[1:], saying[index:])#这行代码是新添的
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail #如果没有模板匹配就返回 [True, None]

这段程序里比较重要的一个新函数是 ```segment_match```，这个函数输入是一个以 ```segment_pattern```开头的模式，尽最大可能进行，匹配到这个*边长*的变量对于的部分。

In [15]:
def segment_match(pattern, saying):
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*', '?')

    if not rest: return (seg_pat, saying), len(saying)    
    
    for i, token in enumerate(saying):#token就是每一个单词
        if rest[0] == token and is_match(rest[1:], saying[(i + 1):]):#rest属于pattern的部分，i应该是从0开始循环，所有要i + 1
            return (seg_pat, saying[:i]), i
    
    return (seg_pat, saying), len(saying)#返回len（）对应的是上一个函数pat_match_with_seg中的index

def is_match(rest, saying):#这段代码是match ?*P 后面部分的单词
    if not rest and not saying:
        return True
    if not all(a.isalpha() for a in rest[0]):
        return True
    if rest[0] != saying[0]:
        return False
    return is_match(rest[1:], saying[1:])

In [180]:
'?*P is very good'.split()[0]

'?*P'

In [181]:
segment_match('?*P is very good'.split(), "My dog and my cat is very good".split())
#注意还返回了句子长度

(('?P', ['My', 'dog', 'and', 'my', 'cat']), 5)

现在，我们就可以做到以下的匹配模式了: 

In [182]:
pat_match_with_seg('?*P is very good and ?*X'.split(), "My dog is very good and my cat is very cute".split())

[('?P', ['My', 'dog']), ('?X', ['my', 'cat', 'is', 'very', 'cute'])]

如果我们继续定义一些模板，我们进行匹配，就能够进行更加复杂的问题了: 

In [358]:
#这里的“？”后面也没有“*”
response_pair = {
    'I need ?X': [
        "Why do you neeed ?X"
    ],
    "I dont like my ?X": ["What bad things did ?X do for you?"]
}

In [152]:
pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())

[('?X', ['an', 'iPhone'])]

In [138]:
#思路：先写个简单的模板
a = 2
c = {1:11,2:22}
for key in c.keys():
    #print(key)
    if a == key:
        print(c[key])

22


In [141]:
#再封装成函数
def get_value(a):
    c = {1:11,2:22}
    for key in c.keys():
    #print(key)
        if a == key:
            return c[key]

In [145]:
get_value(1)
#测试成功！

11

In [144]:
get_value(2)

22

In [207]:
for key in response_pair.keys():
    print(key)

I need ?X
I dont like my ?X


In [221]:
for key in rules.keys():
    print(key)

I was ?*X
?*X hello ?*Y


In [460]:
#自底向上：
import random
choice =random.choice
for key in rules.keys():
    #print(key)
    a = pat_match_with_seg(key.split()  , 
                  'I was an  nice apple'.split())
    #print(a)
    if a  !=[True, None]:
        #print(choice(rules[key]))
        b = "".join(choice(rules[key]))
        #print(b)
        print(" ".join(subsitite(choice(rules[key]).split(), pat_to_dict(pat_match_with_seg(key.split(), 
                      'I was an  nice apple'.split())))))

I already knew you were an nice apple .


In [428]:
subsitite("Why do you neeed ?X".split(), pat_to_dict(pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())))

['Why', 'do', 'you', 'neeed', 'an iPhone']

In [437]:
choice(rules[key])

'Were you really ?X ?'

In [339]:
import random
choice =random.choice

In [408]:
#封装成函数：
def get_response(saying):
    for key in rules.keys():
    #print(key)
        a = pat_match_with_seg(key.split(), 
                  'I was an  nice apple'.split())
    #print(a)
        if a  !=[True, None]:
            
            b = " ".join(subsitite(rules[key].split(), pat_to_dict(pat_match_with_seg(key.split(), 
                      'I was an  nice apple'.split()))))
            return b

In [409]:
get_response('I was an  nice apple')

AttributeError: 'list' object has no attribute 'split'

In [291]:
rules = {
    "I was ?*X": ["Were you really ?X ?", "I already knew you were ?X ."],
     'I need ?*X': ["Why do you neeed ?X"],
    "I dont like my ?*X": ["What bad things did ?X do for you?"]
}

In [355]:
#
pat_match_with_seg('I need ?*X'.split(), 
                  'I need an  nice apple'.split())

[('?X', ['an', 'nice', 'apple'])]

In [356]:
#
" ".join(response_pair['I need ?X'])

'Why do you neeed ?X'

In [372]:
#
" ".join(subsitite("Why do you neeed ?X".split(), pat_to_dict(pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split()))))

'Why do you neeed an iPhone'

In [411]:
subsitite("Why do you neeed ?X".split(), pat_to_dict(pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())))
#subsitite(输出, pat_to_dict(pat_match_with_seg(模板, 
#                  输入)))

['Why', 'do', 'you', 'neeed', 'an iPhone']

 我们会发现，pat_to_dict在这个场景下会有有一点小问题，没关系，修正一些: 

In [16]:
def pat_to_dict(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k, v in patterns}
#isinstance() 函数来判断一个对象是否是一个已知的类型，类似 type()。

In [371]:
subsitite("Why do you neeed ?X".split(), pat_to_dict(pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())))#和往前数第2个cell一模一样

['Why', 'do', 'you', 'neeed', 'an iPhone']

如果我们定义这样的一个模板:

In [91]:
("?*X hello ?*Y", "Hi, how do you do")#给定  "?*X hello ?*Y"  模式的问题，皆返回回答  "Hi, how do you do" 

('?*X hello ?*Y', 'Hi, how do you do')

In [92]:
subsitite("Hi, how do you do?".split(), pat_to_dict(pat_match_with_seg('?*X hello ?*Y'.split(), 
                  "I am mike, hello ".split())))

['Hi,', 'how', 'do', 'you', 'do?']

### 现在是你的时间了

In [17]:
#我们给大家一些例子: 
    
rules = {
    "?*X hello ?*Y": ["Hi, how do you do?"],
    "I was ?*X": ["Were you really ?X ?", "I already knew you were ?X ."]
}

In [18]:
#测试
subsitite("Were you really ?X ? ".split(), pat_to_dict(pat_match_with_seg('I was ?*X'.split(), 
                  "I was stupid".split())))

['Were', 'you', 'really', 'stupid', '?']

### 问题1

编写一个程序, ```get_response(saying, response_rules)```输入是一个字符串 + 我们定义的 rules，例如上边我们所写的 pattern， 输出是一个回答。

In [19]:
import random
choice =random.choice

In [20]:
response_rules =  {
    "I was ?*X": ["Were you really ?X ?", "I already knew you were ?X ."],
     'I need ?*X': ["Why do you neeed ?X"],
    "I dont like my ?*X": ["What bad things did ?X do for you?"]
}

In [579]:
#自底向上：
for key in response_rules.keys():
    print(key)
    a = pat_match_with_seg(key.split()  , 
                  'I was an  nice apple'.split())
    #print(a)
    if a  !=[True, None]:
        #print(choice(rules[key]))
        #b = "".join(choice(rules[key]))
        #print(b)
        c = subsitite(choice(response_rules[key]).split(), pat_to_dict(pat_match_with_seg(key.split(), 
                      'I was an  nice apple'.split())))
        print(" ".join(c))

I was ?*X
Were you really an nice apple ?
I need ?*X
I dont like my ?*X


In [21]:
#封装成函数：
def  get_response(saying,response_rules):
    """" please implement the code, to get the response as followings:"""
    for key in response_rules.keys():
        a = pat_match_with_seg(key.split()  , 
                  saying.split())
        if a  !=[True, None]:
            c = subsitite(choice(response_rules[key]).split(), pat_to_dict(pat_match_with_seg(key.split(), 
                      saying.split())))
            return " ".join(c)
    """>>> get_response('I need iPhone') 
    >>> Image you will get iPhone soon
    >>> get_response("My mother told me something")
    >>> Talk about more about your monther.
    """

In [22]:
#测试成功！
get_response( 'I was an sexy boy',response_rules)

'I already knew you were an sexy boy .'

### 问题2

改写以上程序，将程序变成能够支持中文输入的模式。
*提示*: 你可以需用用到 jieba 分词

In [23]:
import jieba

In [24]:
response_rules2 ={
        "我是?*X": ["你真的是 ?", "我肯定知道你是? ."],
     '我想要?*X': ["为什么你想要?","?对你来说意味着什么呢"],
    "我不喜欢?*X": ["?对你干了什么让你讨厌的事情？"]
}

In [25]:
#自底向上：
for key in response_rules2.keys():
    #print(key)
    a = pat_match_with_seg(jieba.lcut(key)  , 
                 jieba.lcut( '我想要智慧'))
    #print(a)
    if a  !=[True, None]:
        #print(choice(response_rules2[key]))
        c = subsitite(jieba.lcut(choice(response_rules2[key])), pat_to_dict(pat_match_with_seg(jieba.lcut(key), 
                      jieba.lcut( '我想要智慧'))))
        print("".join(c))

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\672\AppData\Local\Temp\jieba.cache
Loading model cost 1.509 seconds.
Prefix dict has been built succesfully.


智慧对你来说意味着什么呢


In [26]:
#封装成函数：
def  get_response(saying,response_rules):
    for key in response_rules2.keys():
    #print(key)
        a = pat_match_with_seg(jieba.lcut(key)  , 
                 jieba.lcut( saying))
    #print(a)
        if a  !=[True, None]:
        #print(choice(response_rules2[key]))
            c = subsitite(jieba.lcut(choice(response_rules2[key])), pat_to_dict(pat_match_with_seg(jieba.lcut(key), 
                      jieba.lcut( saying))))
            return ("".join(c))

In [27]:
 get_response('我想要智慧',response_rules2)

'智慧对你来说意味着什么呢'

### 问题3

多设计一些模式，让这个程序变得更好玩，多和大家交流，看看大家有什么好玩的模式

In [55]:
#制作撩妹聊天机器人
response_rules4 ={
    "这个和听?*X": ["?说送手机一定要送给女朋友"],
}

In [57]:
#自底向上：
for key in response_rules4.keys():
    #print(key)
    a = pat_match_with_seg(jieba.lcut(key)  , 
                 jieba.lcut( '这个和听妈妈'))
    #print(a)
    if a  !=[True, None]:
        #print(choice(response_rules2[key]))
        c = subsitite(jieba.lcut(choice(response_rules4[key])), pat_to_dict(pat_match_with_seg(jieba.lcut(key), 
                      jieba.lcut( '这个和听妈妈'))))
        print("".join(c))

妈妈说送手机一定要送给女朋友


In [58]:
#封装成函数：
def  get_response4(saying,response_rules):
    for key in response_rules4.keys():
    #print(key)
        a = pat_match_with_seg(jieba.lcut(key)  , 
                 jieba.lcut(saying))
    #print(a)
        if a  !=[True, None]:
        #print(choice(response_rules2[key]))
            c = subsitite(jieba.lcut(choice(response_rules4[key])), pat_to_dict(pat_match_with_seg(jieba.lcut(key), 
                      jieba.lcut(saying))))
            return ("".join(c))

In [59]:
get_response4('这个和听妈妈',response_rules4)

'妈妈说送手机一定要送给女朋友'

### 问题4

1. 这样的程序有什么优点？有什么缺点？你有什么可以改进的方法吗？

答：优点简便易行，能够通过规则和模式匹配来灵活修改以满足业务的需要；
缺点是超出一下规则范围机器人就无法回答；改进方法是引入相关NLP深度学习
模型。

2. 什么是数据驱动？数据驱动在这个程序里如何体现？

答：数据驱动就是通过各种手段采集海量的数据，将数据进行整合形成信息，
并加工和提炼，并此基础上训练自动化的决策模型。在这过程中，采集数据、
建立模型、自动决策形成闭环，不断迭代优化；
在这程序里体现在就是我们不断制定新的rule，每碰到新的业务问题就制定
新的rule，从而使机器人更具有鲁棒性。


3. 数据驱动与 AI 的关系是什么？ 

答：相互促进，相互发展。AI的基础是数据，依靠数据来驱动AI模型，数据是AI
的源泉。AI是数据的有效工具，没有AI，数据中的信息就没办法得到挖掘，数据
的价值及没办法体现。

一些参考 pattern

In [89]:
rule_responses = {
    '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    '?*x I want ?*y': ['what would it mean if you got ?y', 'Why do you want ?y', 'Suppose you got ?y soon'],
    '?*x if ?*y': ['Do you really think its likely that ?y', 'Do you wish that ?y', 'What do you think about ?y', 'Really-- if ?y'],
    '?*x no ?*y': ['why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?'],
    '?*x I was ?*y': ['Were you really', 'Perhaps I already knew you were ?y', 'Why do you tell me you were ?y now?'],
    '?*x I feel ?*y': ['Do you often feel ?y ?', 'What other feelings do you have?'],
    '?*x你好?*y': ['你好呀', '请告诉我你的问题'],
    '?*x我想?*y': ['你觉得?y有什么意义呢？', '为什么你想?y', '你可以想想你很快就可以?y了'],
    '?*x我想要?*y': ['?x想问你，你觉得?y有什么意义呢?', '为什么你想?y', '?x觉得... 你可以想想你很快就可以有?y了', '你看?x像?y不', '我看你就像?y'],
    '?*x喜欢?*y': ['喜欢?y的哪里？', '?y有什么好的呢？', '你想要?y吗？'],
    '?*x讨厌?*y': ['?y怎么会那么讨厌呢?', '讨厌?y的哪里？', '?y有什么不好呢？', '你不想要?y吗？'],
    '?*xAI?*y': ['你为什么要提AI的事情？', '你为什么觉得AI要解决你的问题？'],
    '?*x机器人?*y': ['你为什么要提机器人的事情？', '你为什么觉得机器人要解决你的问题？'],
    '?*x对不起?*y': ['不用道歉', '你为什么觉得你需要道歉呢?'],
    '?*x我记得?*y': ['你经常会想起这个吗？', '除了?y你还会想起什么吗？', '你为什么和我提起?y'],
    '?*x如果?*y': ['你真的觉得?y会发生吗？', '你希望?y吗?', '真的吗？如果?y的话', '关于?y你怎么想？'],
    '?*x我?*z梦见?*y':['真的吗? --- ?y', '你在醒着的时候，以前想象过?y吗？', '你以前梦见过?y吗'],
    '?*x妈妈?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '她对你影响很大吗？'],
    '?*x爸爸?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '他对你影响很大吗？', '每当你想起你爸爸的时候， 你还会想起其他的吗?'],
    '?*x我愿意?*y': ['我可以帮你?y吗？', '你可以解释一下，为什么想?y'],
    '?*x我很难过，因为?*y': ['我听到你这么说， 也很难过', '?y不应该让你这么难过的'],
    '?*x难过?*y': ['我听到你这么说， 也很难过',
                 '不应该让你这么难过的，你觉得你拥有什么，就会不难过?',
                 '你觉得事情变成什么样，你就不难过了?'],
    '?*x就像?*y': ['你觉得?x和?y有什么相似性？', '?x和?y真的有关系吗？', '怎么说？'],
    '?*x和?*y都?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x和?*y一样?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x我是?*y': ['真的吗？', '?x想告诉你，或许我早就知道你是?y', '你为什么现在才告诉我你是?y'],
    '?*x我是?*y吗': ['如果你是?y会怎么样呢？', '你觉得你是?y吗', '如果你是?y，那一位着什么?'],
    '?*x你是?*y吗':  ['你为什么会对我是不是?y感兴趣?', '那你希望我是?y吗', '你要是喜欢， 我就会是?y'],
    '?*x你是?*y' : ['为什么你觉得我是?y'],
    '?*x因为?*y' : ['?y是真正的原因吗？', '你觉得会有其他原因吗?'],
    '?*x我不能?*y': ['你或许现在就能?*y', '如果你能?*y,会怎样呢？'],
    '?*x我觉得?*y': ['你经常这样感觉吗？', '除了到这个，你还有什么其他的感觉吗？'],
    '?*x我?*y你?*z': ['其实很有可能我们互相?y'],
    '?*x你为什么不?*y': ['你自己为什么不?y', '你觉得我不会?y', '等我心情好了，我就?y'],
    '?*x好的?*y': ['好的', '你是一个很正能量的人'],
    '?*x嗯嗯?*y': ['好的', '你是一个很正能量的人'],
    '?*x不嘛?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x不要?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x有些人?*y': ['具体是哪些人呢?'],
    '?*x有的人?*y': ['具体是哪些人呢?'],
    '?*x某些人?*y': ['具体是哪些人呢?'],
    '?*x每个人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x所有人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x总是?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x一直?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x或许?*y': ['你看起来不太确定'],
    '?*x可能?*y': ['你看起来不太确定'],
    '?*x他们是?*y吗？': ['你觉得他们可能不是?y？'],
    '?*x': ['很有趣', '请继续', '我不太确定我很理解你说的, 能稍微详细解释一下吗?']
}
